# Regresión KNN
---

Estableciendo el lugar donde se encuentra la data, en nuestro caso lo almacenamos en una carpeta en Drive:

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/AIProjects/PC2/Data/PracticaCalificada2-CC421/
%ls

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/.shortcut-targets-by-id/1ouCzEh7A4-s7GMx2btcWWTXkJxKP-98c/AIProjects/PC2/Data/PracticaCalificada2-CC421
data2.csv                          train_2014.csv
data2.csv.hdf5                     train_2014_processed.csv
data2.csv.yaml                     train_2015.csv
data.csv                           train_2015_processed.csv
ht/                                train_2.csv
PracticaCalificada2-CC421.pdf      train_2_processed.csv
rf_0                               train_2_processed_wo_gd.csv
rf_1                               train_3.csv
rf_2                               train_3_processed.csv
rf_3                               train_3_processed_wo_gd.csv
rf_4                               train_4.csv
train_0.csv                        train_4_processed.csv
train_0_processed.csv              train_4_processed_wo_gd.csv
train_0_processed_wo_gd.csv        train.csv
train_1

## Modelamiento

#### Importación de librerías necesarias para el entrenamiento

In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Carga de datos para el entrenamiento

In [4]:
df = pd.read_csv('train_data.csv')

In [ ]:
df.head()

#### Selección de características necesarias para el entrenamiento.

El algoritmo kNN requiere de más esfuerzo computacional mientras más características se le agrega. Debido a ello, vamos a usar las características iniciales del dataset y dos características adicionales: `day` (día de la semana) y `hour` (hora del día).

Recordando además que nuestro objetivo es predecir `fare_amout` (tarifa).

In [5]:
features = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
            'dropoff_longitude', 'passenger_count', 'day', 'hour']
target = ['fare_amount']
X = df[features]
y = df[target]

#### División de los datos entre data de entrenamiento y data de prueba

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    random_state=42)

#### Ajustando los parámetros

Cuando el algoritmo kNN va a operar sobre un dataset con muchas características se prefiere usar la métrica `minkowski` en el algoritmo. En nuestro caso usamos 7 características, y para corrobar lo dicho anteriormente realizaremos un `GridSearch` sobre un `k` fijo, tomando los pesos de cada punto de forma `uniform` y `distance`, y las métricas `manhattan`, `euclidean` y `minkowski`.

In [ ]:
gs_params = {
    'n_neighbors': [5],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}
gs = GridSearchCV(KNeighborsRegressor(), gs_params, verbose=4, cv=2, n_jobs=6)
gs_results = gs.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   5 out of  12 | elapsed: 20.9min remaining: 29.2min
[Parallel(n_jobs=6)]: Done   9 out of  12 | elapsed: 42.6min remaining: 14.2min
[Parallel(n_jobs=6)]: Done  12 out of  12 | elapsed: 49.1min finished


In [ ]:
gs_results.best_params_

{'metric': 'minkowski', 'n_neighbors': 5, 'weights': 'distance'}

Tal y como se esperaba para la mejor métrica para la distancia es `minkowski` y tomando como peso `distance`.

Ahora buscaremos el mejor `k` para nuestro modelo kNN fijando `weights='minkowski'` y `metric='distance'`.

In [ ]:
gs_params = {
    'n_neighbors': [3, 4, 5, 6, 7, 8],
    'weights': ['distance'],
    'metric': ['minkowski']
}
gs = GridSearchCV(KNeighborsRegressor(), gs_params, verbose=4, cv=2, n_jobs=6)
gs_results1 = gs.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   5 out of  12 | elapsed: 20.9min remaining: 29.2min
[Parallel(n_jobs=6)]: Done   9 out of  12 | elapsed: 42.6min remaining: 14.2min
[Parallel(n_jobs=6)]: Done  12 out of  12 | elapsed: 49.1min finished


In [ ]:
gs_results1.best_params_

{'metric': 'minkowski', 'n_neighbors': 5, 'weights': 'distance'}

In [ ]:
gs_results1.best_score_

0.7900549440850198

Finalmente obtuvimos que los mejores parámetros para nuestro modelo son:
- `k = 5`
- `weights = 'distance'`
- `metric = 'minkowski'`

#### Entrenamiento del Modelo con los Parámetros Optimizados

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5, weights='distance', metric='minkowski',
                          n_jobs=6)
knn.fit(X_train, y_train)
print(f"Score en entrenamiento: {knn.score(X_train, y_train)}")

Score en entrenamiento: 0.9999979589641794


Notamos que el modelo presentó un sobreajuste en el entrenamiento. Ahora mostraremos el score y otras métricas en la prueba.

In [ ]:
y_pred = knn.predict(X_test)
print(f"Score: {knn.score(X_test, y_test)}")
print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False)}")

Score: 0.810235209882544
RMSE: 4.501953305762435


En la prueba el modelo tuvo un 81% de precisión y un RMSE de 4.5.